In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [16]:
!pwd

/t1-data/project/fgenomics/jinchaus/rotation


In [2]:
df = pd.read_csv("/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_3groups_K27_ATAC.csv", sep = "\t")
df.shape

data = np.array(df[df.columns.tolist()[2:]])

data_zscore = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
labels = np.array(df[df.columns.tolist()[0]])


In [3]:
df['group'].value_counts()

LoF      40
LoK27    40
other    36
GoF      28
Name: group, dtype: int64

In [ ]:
# Split dataset into training set and test set
#X_train, X_test, y_train, y_test = train_test_split(data_zscore, labels , test_size=0.1,random_state=100) 


In [4]:
y_train = labels
y_train.shape

(144,)

In [5]:
X_train = data_zscore
X_train.shape

(144, 100)

In [6]:
np.unique(y_train, return_counts=True)

(array(['GoF', 'LoF', 'LoK27', 'other'], dtype=object),
 array([28, 40, 40, 36]))

In [7]:
ML_results = pd.DataFrame()

In [8]:
from sklearn import svm
clf = make_pipeline(StandardScaler(), SVC(gamma="auto", degree = 3, kernel = "rbf")) # add probability = True for k-fold cross val
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X_train, y_train, cv=5)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

scores = np.append(scores, scores.mean())
scores = np.append(scores,scores.std())

ML_results['SVM']= scores

0.65 accuracy with a standard deviation of 0.07


In [9]:
scores

array([0.5862069 , 0.72413793, 0.55172414, 0.72413793, 0.64285714,
       0.64581281, 0.06414038])

In [10]:
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.pipeline import Pipeline
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=3)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])

scores = cross_val_score(nca_pipe, X_train, y_train, cv=5)

print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
scores = np.append(scores, scores.mean())
scores = np.append(scores,scores.std())
ML_results['NCA + KN']= scores

[0.5862069  0.75862069 0.68965517 0.75862069 0.64285714]
0.69 accuracy with a standard deviation of 0.07


In [11]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)

scores = cross_val_score(clf, X_train, y_train, cv=5)

print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
scores = np.append(scores, scores.mean())
scores = np.append(scores,scores.std())
ML_results['Gaussian NB']= scores

[0.44827586 0.48275862 0.5862069  0.44827586 0.57142857]
0.51 accuracy with a standard deviation of 0.06


In [12]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
scores = cross_val_score(clf, X_train, y_train, cv=5)

print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

X_train.shape
scores = np.append(scores, scores.mean())
scores = np.append(scores,scores.std())
ML_results['MLP']= scores

/package/python-cbrg/current/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/package/python-cbrg/current/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/package/python-cbrg/current/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs

[0.55172414 0.55172414 0.55172414 0.51724138 0.60714286]
0.56 accuracy with a standard deviation of 0.03


/package/python-cbrg/current/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [13]:
ML_results.rename(index={0:'Fold 1', 1:'Fold 2', 2:'Fold 3', 3:'Fold 4', 4:'Fold 5',5:'Mean', 6:'SD'}, inplace=True)

In [14]:
ML_results = ML_results.round(decimals = 2)

In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

display(ML_results)

,SVM,NCA + KN,Gaussian NB,MLP
Fold 1,0.59,0.59,0.45,0.55
Fold 2,0.72,0.76,0.48,0.55
Fold 3,0.55,0.69,0.59,0.55
Fold 4,0.72,0.76,0.45,0.52
Fold 5,0.64,0.64,0.57,0.61
Mean,0.65,0.69,0.51,0.56
SD,0.06,0.06,0.05,0.03


# Deep learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
#!module load tensorflow2-cpu/202102 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold, StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os, random

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
def plot_history(hist):
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc=4)
    plt.savefig('/datashare/project/fgenomics/jinchaus/rotation/plots/Fold_Epochs_Acc.png', dpi =400)
    plt.show()
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper right')  
    plt.savefig('/datashare/project/fgenomics/jinchaus/rotation/plots/Fold_Epochs_Loss.png', dpi =400)
    plt.show()

In [ ]:
df = pd.read_csv("/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_3groups_K27_ATAC.csv", sep = "\t")
data = np.array(df[df.columns.tolist()[2:]])

data_zscore = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
labels = np.array(df[df.columns.tolist()[0]])
                 
                 

In [ ]:
df['group'].value_counts()

In [ ]:
# prepare target
le = LabelEncoder()
le.fit(labels)
labels_enc = le.transform(labels)
labels_enc = to_categorical(labels_enc)

In [ ]:
#with k-fold validation

mean_accuracy=[]
skf = StratifiedKFold(n_splits = 5, random_state = 1, shuffle = True) 
for j, (train_idx, val_idx) in enumerate(skf.split(data_zscore, labels)):

    print('\nFold ',j)
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]

    X_train = data_zscore[train_idx]
    y_train = labels_enc[train_idx]
    X_test = data_zscore[val_idx]
    y_test= labels_enc[val_idx]

    tf.random.set_seed(42)
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_shape=(100,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(3, activation="sigmoid"),])

#     model = keras.Sequential([
#         layers.Dense(64, activation="relu", input_shape=(100,)),
#         layers.Dense(32, activation="relu"),
#         layers.Dense(4, activation="sigmoid"),])
    
#     model = keras.Sequential([
#         layers.Dense(256, activation="sigmoid", input_shape=(100,)),
#         layers.Dense(128, activation="sigmoid"),
#         layers.Dense(32, activation="sigmoid"),
#         layers.Dense(4, activation="sigmoid"),])

    model.compile(
        optimizer=keras.optimizers.Adam(),
        #optimizer=keras.optimizers.RMSprop(),
        loss='binary_crossentropy',
        metrics=['acc'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

    
    best_model = tf.keras.callbacks.ModelCheckpoint(
        filepath='k_fold_best_'+str(j)+'_4groups.h5',
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    print("Fit model on training data")
    history = model.fit(X_train, y_train, epochs = 500, validation_data=(X_test, y_test), 
                        verbose=0, callbacks=[early_stopping, best_model])
    #plot_history(history)

    #model.load_weights('k_fold_best.h5')

    print("Evaluate on test data")
    results = model.evaluate(X_test, y_test)
    mean_accuracy.append(results[1])
    print("test loss, test acc:", results)
    #os.remove('k_fold_best.h5')

print("Mean accuracy from 5 folds: ", np.mean(mean_accuracy))




In [ ]:
np.std(mean_accuracy)

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(data_zscore,
                                                    labels_enc, 
                                                    test_size=0.2,
                                                    stratify=labels_enc,
                                                    random_state=100)

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu", input_shape=(100,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(3, activation="sigmoid"),
    ]
)


In [ ]:
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    # Loss function to minimize
    loss='binary_crossentropy',
    # List of metrics to monitor
    metrics=['acc'],
)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=50, restore_best_weights= True)
print("Fit model on training data")
history = model.fit(X_train, y_train, epochs = 500, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping, best_model])

# Probably better to train less epochs. 500 right now. try 150

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
le.inverse_transform(np.argmax(prediction, axis=1))

In [ ]:
le.inverse_transform(np.argmax(y_test, axis=1))

In [ ]:
y_true=le.inverse_transform(np.argmax(y_test, axis=1))
y_pred=le.inverse_transform(np.argmax(prediction, axis=1))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pylab as pl
cm = confusion_matrix(y_true, y_pred, labels=["GoF", "LoF","other"])
#disp.matshow(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=["GoF", "LoF", "other"])

disp.plot()

disp.figure_.savefig('/datashare/project/fgenomics/jinchaus/rotation/plots/Fold_confusion_matrix.png', dpi =400)



In [ ]:
plot_history(history)

# Adding Loss of K27

In [ ]:
All = pd.read_csv("/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_4groups_K27_ATAC.csv", sep = "\t")

In [ ]:
K27 = All[All['group'] == "LoK27"]

In [ ]:
data = np.array(K27[K27.columns.tolist()[2:]])

data_zscore = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
labels = np.array(K27[K27.columns.tolist()[1]])
                 

In [ ]:
data_zscore.shape

In [ ]:
prediction = model.predict(data_zscore)

In [ ]:
k27 = le.inverse_transform(np.argmax(prediction, axis=1))
np.unique(k27, return_counts=True)


In [ ]:
Lo = All[(All['group'] == "LoK27")|(All['group'] == "other")]
#Lo = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_2groups_K27_LoK27.csv', sep = "\t")
data = np.array(Lo[Lo.columns.tolist()[2:]])
data_zscore = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
#sx=data_zscore.T[:50].T
#dx=data_zscore.T[50:].T
#data_zscore = pd.DataFrame(np.divide(sx, dx, out = sx, where=dx!=0))

#data_zscore = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
labels = np.array(Lo[Lo.columns.tolist()[0]])
                 

data_zscore.shape

In [ ]:
# prepare target
le = LabelEncoder()
le.fit(labels)
labels_enc = le.transform(labels)
labels_enc = to_categorical(labels_enc)

In [ ]:

mean_accuracy=[]
skf = StratifiedKFold(n_splits = 5, random_state = 1, shuffle = True) 
for j, (train_idx, val_idx) in enumerate(skf.split(data_zscore, labels)):

    print('\nFold ',j)
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]

    X_train = data_zscore[train_idx]
    y_train = labels_enc[train_idx]
    X_test = data_zscore[val_idx]
    y_test= labels_enc[val_idx]

    tf.random.set_seed(42)
    model = keras.Sequential([
        layers.Dense(16, activation="relu", input_shape=(100,)),
        layers.Dense(2, activation="sigmoid"),])

    model.compile(
        optimizer=keras.optimizers.Adam(),
        #optimizer=keras.optimizers.RMSprop(),
        loss='binary_crossentropy',
        metrics=['acc'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

    
    best_model = tf.keras.callbacks.ModelCheckpoint(
        filepath='k_fold_best_'+str(j)+'_2groups.h5',
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    print("Fit model on training data")
    history = model.fit(X_train, y_train, epochs = 500, validation_data=(X_test, y_test), 
                        verbose=0, callbacks=[early_stopping, best_model])
    #plot_history(history)

    #model.load_weights('k_fold_best.h5')

    print("Evaluate on test data")
    results = model.evaluate(X_test, y_test)
    mean_accuracy.append(results[1])
    print("test loss, test acc:", results)
    #os.remove('k_fold_best.h5')

print("Mean accuracy from 5 folds: ", np.mean(mean_accuracy))



In [ ]:
np.std(mean_accuracy)

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(data_zscore,
                                                    labels_enc, 
                                                    test_size=0.3,
                                                    stratify=labels_enc,
                                                    random_state=100)

In [ ]:
# Define Sequential model with 2 layers
model = keras.Sequential(
    [
        layers.Dense(16, activation="relu", input_shape=(100,)),
        #layers.Dense(32, activation="relu"),
        #layers.Dense(8, activation="sigmoid"),
        layers.Dense(2, activation="sigmoid"),
    ]
)


In [ ]:
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),
    #optimizer=keras.optimizers.SGD(),
    # Loss function to minimize
    loss='binary_crossentropy',
    # List of metrics to monitor
    metrics=['acc'],
)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=50, restore_best_weights= True)
print("Fit model on training data")
history = model.fit(X_train, y_train, epochs = 500, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping])
plot_history(history)
# Probably better to train less epochs. 500 right now. try 150

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
le.inverse_transform(np.argmax(prediction, axis=1))

In [ ]:
le.inverse_transform(np.argmax(y_test, axis=1))

In [ ]:
y_true=le.inverse_transform(np.argmax(y_test, axis=1))
y_pred=le.inverse_transform(np.argmax(prediction, axis=1))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pylab as pl
cm = confusion_matrix(y_true, y_pred, labels=["other", "LoK27"])
#disp.matshow(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=["other", "LoK27"])

disp.plot()

